In [ ]:
# Dependencies go in this cell.
# The script env.sh in this directory should create an Anaconda environment with
# all these dependencies installed

# Python built-in packages
import functools
import importlib
from typing import *

# Libraries
import numpy as np
import pandas as pd
import regex
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

# TEMPORARY until we can use Python 3.8 functools' built-in memoized property
from memoized_property import memoized_property

#TEST_DOC_FILE = "resources/example_doc.txt"
TEST_DOC_FILE = "resources/short_example_doc.txt"

with open (TEST_DOC_FILE, "r") as f:
    TEST_TEXT = f.read()


# [re]import our local library code. Usually doesn't work, but worth a try.
import pandas_text as pt
importlib.reload(pt)

In [ ]:
class Resources:
    """
    Data structures that are loaded once, as opposed to recreated on
    every document.
    
    This category includes tokenizers, dictionaries, and compiled regexes.
    
    Everything in this class is a cached property
    """
    
    def preload_all(self):
        """
        Preload all cached values in this class so it won't be necessary to 
        load them lazily later on.
        """
        for name in dir(self):
            if name[0].isupper():
                getattr(self, name)
                
    @memoized_property
    def LanguageModel(self):
        return spacy.load("en_core_web_sm")
    
    @memoized_property
    def Tokenizer(self):
        nlp = self.LanguageModel
        return nlp.Defaults.create_tokenizer(nlp)
        
    
    ############################
    # DICTIONARIES
    
    @memoized_property
    def GlobalFirstNameDict(self):
        # Use the "pre-compiled" Parquet version of the dictionary, which we
        # produced by running:
        #    pt.load_dict("resources/first_name.dict", self.Tokenizer) \
        #      .to_parquet("resources/first_name.dict.parquet")
        return pd.read_parquet("resources/first_name.dict.parquet")
    
    @memoized_property
    def GlobalLastNameDict(self):
        return pd.read_parquet("resources/last_name.dict.parquet")
    
    ############################
    # REGEXES
    
    @memoized_property
    def CapsWordRegex(self):
        return regex.compile("[A-Z][a-z]*")
       
resources = Resources()
resources.preload_all()
#resources.GlobalFirstNameDict[~resources.GlobalFirstNameDict["toks_1"].isna()]
#resources.GlobalFirstNameDict

In [ ]:
class Document:
    """
    Define the fields of the current document.
    
    In this case the document is a single string.
    """
    def __init__(self, doc_text: str, resources: Resources):
        self._text = doc_text
        self._resources = resources
        
    @property
    def Text(self):
        return self._text
    
    @memoized_property
    def TokenFeatures(self):
        return pt.make_tokens_and_features(self._text, self._resources.LanguageModel)
    
    @memoized_property
    def Sentence(self):
        return pd.DataFrame({"sentence": self.TokenFeatures["sentence"].unique()})
    
    @property
    def Tokens(self):
        """
        :return: tokens as a `pd.Series` backed by a `CharSpanArray`.
        """
        return self.TokenFeatures["char_span"]
    
    

In [ ]:
# Create an example Document object and show the Tokens as a DataFrame
doc = Document(TEST_TEXT, resources)
doc.TokenFeatures

In [ ]:
# Unique spans from the "sentence" column in doc.TokenFeatures
doc.Sentence

In [ ]:
# Contents of the seventh sentence in the document
seventh_sentence = doc.TokenFeatures[doc.TokenFeatures["sentence"] == doc.Sentence["sentence"][6]]
seventh_sentence

In [ ]:
# Show the dependency parse of the sentence
pt.render_parse_tree(seventh_sentence)

In [ ]:
# Pretty-print the tokens in the sentence
seventh_sentence["token_span"].values

In [ ]:
# Use a Gremlin query to find all compound nouns in the sentence
g = pt.token_features_to_traversal(seventh_sentence)
compound_nouns = (
    g.V()
    .has("dep", "compound").as_("src")
    .out().as_("dest")
    .select("src", "dest").by("token_span")
).toDataFrame()
compound_nouns

In [ ]:
# Filter down the seventh sentence to just the tokens that take part in compound nouns
all_tokens_df = pd.DataFrame({
    "token_span" : pd.concat([compound_nouns[c] for c in compound_nouns]).unique()})
compound_noun_tokens = seventh_sentence.merge(all_tokens_df)
compound_noun_tokens = compound_noun_tokens.set_index(compound_noun_tokens["token_num"])
compound_noun_tokens

In [ ]:
# Render the partial parse trees of just those tokens
pt.render_parse_tree(compound_noun_tokens)

In [ ]:
# Build some business rules that define some text features.
# The rules are organized into Python classes.
# The output of each rule is a Pandas DataFrame.

class Dictionaries:
    """
    Rules that evaluate dictionaries against the document's raw tokens.
    """
    def __init__(self, d: Document, resources: Resources):
        self._d = d
        self._resources = resources
    
    @memoized_property
    def GlobalFirstName(self):
        return pt.extract_dict(self._d.Tokens, self._resources.GlobalFirstNameDict)
    
    @memoized_property
    def GlobalLastName(self):
        return pt.extract_dict(self._d.Tokens, self._resources.GlobalLastNameDict)

class Regexes:
    """
    Rules that evaluate regular expressions against the document's raw tokens.
    """
    def __init__(self, d: Document, resources: Resources):
        self._d = d
        self._resources = resources
    
    @property
    def CapsWord(self):
        """
        A single token that starts with a capital letter, with subsequent letters not
        capitalized.
        """
        return pt.extract_regex_tok(
            tokens = self._d.Tokens,
            compiled_regex = self._resources.CapsWordRegex)
    

class Morphology:
    """
    Rules that filter tokens according to shallow linguistic features.
    """
    def __init__(self, d: Document):
        self._d = d
        
    @property
    def ProperNounToken(self):
        """
        Tokens that the part of speech tagger tagged as proper nouns.
        """
        feats = self._d.TokenFeatures
        return pd.DataFrame({"match": feats["token_span"][feats["tag"] == "NNP"]})



In [ ]:
# Show the tokens labeled as proper nouns
doc = Document(TEST_TEXT, resources)
morph = Morphology(doc)
morph.ProperNounToken

In [ ]:
# Pretty-print the spans in ProperNounToken
morph.ProperNounToken["match"].values

In [ ]:
# Write some additional business rules that define a person extractor.
# Note the use of a Python method to avoid duplicate code in the rules.
    
class PersonName:
    """
    Rules that extract potential person name entities.
    """
    def __init__(self, doc: Document, dicts: Dictionaries, regexes: Regexes,
                 morphology: Morphology):
        self._doc = doc
        self._dicts = dicts
        self._regexes = regexes
        self._morphology = morphology

    @staticmethod
    def first_last_name(first: pd.DataFrame, last: pd.DataFrame):
        """
        Generic <first name> <last name> pattern match. Subroutine of rules below.
        
        :param first: DataFrame of first names, with the name in the column "match".
        
        :param last: DataFrame of last names, with the name in the column "match".
        
        :returns: A DataFrame with all <first name> <last name> matches, including the
            columns "first_name", "last_name", and "name" 
            (span that covers both first and last names)
        """
        ret = pt.adjacent_join(
            first_series = first["match"],
            second_series = last["match"],
            first_name = "first_name",
            second_name = "last_name")
        ret["name"] = pt.combine_spans(ret["first_name"], ret["last_name"])
        return ret
    
    @property
    def Person1(self):
        """
        <match of GlobalFirstName dict> <match of GlobalLastName dict>
        """
        return PersonName.first_last_name(self._dicts.GlobalFirstName, self._dicts.GlobalLastName)
    
    @property
    def Person2(self):
        """
        <match of GlobalFirstName dict> <capitalized word>
        """
        return PersonName.first_last_name(self._dicts.GlobalFirstName, self._regexes.CapsWord)
    
    @property
    def Person3(self):
        """
        <token labeled as proper noun> <match of GlobalLastName dict>
        """
        return PersonName.first_last_name(self._morphology.ProperNounToken, self._dicts.GlobalLastName)


In [ ]:
# Instantiate our rules for a document
doc = Document(TEST_TEXT, resources)
dicts = Dictionaries(doc, resources)
regexes = Regexes(doc, resources)
morph = Morphology(doc)
persons = PersonName(doc, dicts, regexes, morph)

In [ ]:
# Show one of the output DataFrames
persons.Person3

In [ ]:
# Show a detailed view of the "name" column of the above DataFrame
persons.Person3["name"].values